In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [2]:
#Following is the code to load required libraries and data:
import numpy as np
import pandas as pd

train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)

test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test' , skiprows = 1, header = None)

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num','marital_status', 'occupation','relationship', 'race', 'sex', 'capital_gain','capital_loss', 'hours_per_week', 'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [3]:
df=pd.concat([train_set,test_set])
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [4]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
wage_class        object
dtype: object

In [5]:
cat=[]
for  i in df.columns:
    if(df[i].dtype=='object'):
        cat.append(i)
cat

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'wage_class']

In [6]:
df[cat].describe()

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,wage_class
count,48842,48842,48842,48842,48842,48842,48842,48842,48842
unique,9,16,7,15,6,5,2,42,4
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,33906,15784,22379,6172,19716,41762,32650,43832,24720


In [7]:
###CHECKING THE unique_value AND count of each featurehave object datatype
for i in df.columns:
    if (df[i].dtype=='object'):
        print('_'*15)
        print(i)
        print(''*15)
        print(df[i].value_counts())
    

_______________
workclass

 Private             33906
 Self-emp-not-inc     3862
 Local-gov            3136
 ?                    2799
 State-gov            1981
 Self-emp-inc         1695
 Federal-gov          1432
 Without-pay            21
 Never-worked           10
Name: workclass, dtype: int64
_______________
education

 HS-grad         15784
 Some-college    10878
 Bachelors        8025
 Masters          2657
 Assoc-voc        2061
 11th             1812
 Assoc-acdm       1601
 10th             1389
 7th-8th           955
 Prof-school       834
 9th               756
 12th              657
 Doctorate         594
 5th-6th           509
 1st-4th           247
 Preschool          83
Name: education, dtype: int64
_______________
marital_status

 Married-civ-spouse       22379
 Never-married            16117
 Divorced                  6633
 Separated                 1530
 Widowed                   1518
 Married-spouse-absent      628
 Married-AF-spouse           37
Name: marital_statu

In [8]:
#obseerved that '?' is there in three columns ----->workclass,occupation and native_country

In [9]:
ind=df[df['workclass']==" ?"].index
print(df.shape,"before removing index")
df.drop(ind,axis=0,inplace=True)
print(df.shape,"After removing index of workclass")


(48842, 15) before removing index
(44260, 15) After removing index of workclass


In [10]:
ind=df[df['occupation']==" ?"].index
print(df.shape,"before removing index")
df.drop(ind,axis=0,inplace=True)
print(df.shape,"After removing index of occupation")

(44260, 15) before removing index
(44244, 15) After removing index of occupation


In [11]:
ind=df[df['native_country']==" ?"].index
print(df.shape,"before removing index")
df.drop(ind,axis=0,inplace=True)
print(df.shape,"After removing index of native_country")

(44244, 15) before removing index
(42960, 15) After removing index of native_country


In [12]:
####checking again for ( ?) values
###CHECKING THE unique_value AND count of each featurehave object datatype
for i in df.columns:
    if (df[i].dtype=='object'):
        print('_'*15)
        print(i)
        print(''*15)
        print(df[i].value_counts())

_______________
workclass

 Private             31661
 Self-emp-not-inc     3609
 Local-gov            2949
 State-gov            1837
 Self-emp-inc         1568
 Federal-gov          1318
 Without-pay            18
Name: workclass, dtype: int64
_______________
education

 HS-grad         14042
 Some-college     9430
 Bachelors        7181
 Masters          2382
 Assoc-voc        1856
 11th             1544
 Assoc-acdm       1432
 10th             1166
 7th-8th           780
 Prof-school       732
 9th               638
 12th              554
 Doctorate         515
 5th-6th           427
 1st-4th           213
 Preschool          68
Name: education, dtype: int64
_______________
marital_status

 Married-civ-spouse       20024
 Never-married            13868
 Divorced                  5997
 Separated                 1321
 Widowed                   1198
 Married-spouse-absent      520
 Married-AF-spouse           32
Name: marital_status, dtype: int64
_______________
occupation

 Craft-rep

In [13]:
####CATEGORICAL TO DUMMY VARIABLES
df1=pd.get_dummies(df[cat[:-1]],drop_first=True)
df1.shape

(42960, 90)

In [14]:
int_col=[]
for i in df.columns:
    if(df[i].dtype!='object'):
        int_col.append(i)
int_col

['age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week']

In [15]:
df2=pd.concat([df[int_col],df1],axis=1)

In [16]:
###CONVERTING TARGET CATEGORICAL VARIABLE INTO SINGLE FEATURE USING LABEL_ENCODER
 
le = preprocessing.LabelEncoder() 
le.fit_transform(df['wage_class'])

array([0, 0, 0, ..., 1, 1, 3])

In [17]:
le.classes_

array([' <=50K', ' <=50K.', ' >50K', ' >50K.'], dtype=object)

In [19]:
test_set

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.
